# This is testing for the chosen Quantile Regression Models
- Based on AIC, BIC scores: Q025_pb0_e ; Q075_pb1_g
- Based on allocations: Q075_pb0_e ; Q075_pb0_g

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, validation_curve
import os
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.preprocessing import StandardScaler
import time
import seaborn as sns
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import log_loss, d2_pinball_score, get_scorer_names
from sklearn.utils import resample

from sklearn.metrics import mean_absolute_error

import math

import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.metrics import mean_squared_error, mean_pinball_loss, make_scorer, accuracy_score, balanced_accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support, r2_score

from google.colab import drive
from os import listdir
from os.path import isfile, join
from joblib import dump, load


from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV, LeaveOneOut
from pprint import pprint


drive.mount('/content/drive')

PATH = '/content/drive/MyDrive/nda'

Mounted at /content/drive


In [ ]:
# Loading the models

split_data_path = PATH + '/split_data/'

E_train = np.loadtxt(join(split_data_path, 'E_train.out'), delimiter=',')
G_train = np.loadtxt(join(split_data_path,'G_train.out'), delimiter=',')
E_test = np.loadtxt(join(split_data_path,'E_test.out'), delimiter=',')
G_test = np.loadtxt(join(split_data_path,'G_test.out'), delimiter=',')

e_train_snr = np.loadtxt(join(split_data_path,'e_train_snr.out'), delimiter=',')
g_train_snr = np.loadtxt(join(split_data_path,'g_train_snr.out'), delimiter=',')
e_test_snr = np.loadtxt(join(split_data_path,'e_test_snr.out'), delimiter=',')
g_test_snr = np.loadtxt(join(split_data_path,'g_test_snr.out'), delimiter=',')

# Chosen models for testing

model_path = PATH + '/models/'

# Based on AIC, BIC scores
Q025_pb0_e = load(join(model_path, 'Q0.25_pb0.00_e.joblib'))
Q075_pb1_g = load(join(model_path, 'Q0.75_pb1.00_g.joblib'))

# Based on allocations
Q075_pb0_e = load(join(model_path, 'Q0.75_pb0.00_e.joblib'))
Q075_pb0_g = load(join(model_path, 'Q0.75_pb0.00_g.joblib'))

In [ ]:
boundries = [8.7, 12.8, 15.2, 18.2, 21.0]

def SNR_to_MF(SNR):
    epsilon = 1e-6  # Small epsilon value for comparison

    if SNR < boundries[0] - epsilon:
        return 0

    elif (boundries[0] - epsilon <= SNR) and (SNR < boundries[1]):
        return 4

    elif (boundries[1] <= SNR) and (SNR < boundries[2]):
        return 8

    elif (boundries[2] <= SNR) and (SNR < boundries[3]):
        return 16

    elif (boundries[3] <= SNR) and (SNR < boundries[4]):
        return 32

    else:  # SNR >= 23.7:
        return 64

# Functions ready to use for the evaluation

In [ ]:
PENALIZE_OVER = 0
PENALIZE_UNDER = 1

def eval_model(y_true, y_pred, alpha):
  mse = mean_squared_error(y_true, y_pred)
  pbl = mean_pinball_loss(y_true, y_pred, alpha=0.5)
  pbo = mean_pinball_loss(y_true, y_pred, alpha=PENALIZE_OVER)
  pbu = mean_pinball_loss(y_true, y_pred, alpha=PENALIZE_UNDER)
  pba = mean_pinball_loss(y_true, y_pred, alpha=alpha)
  mae = mean_absolute_error(y_true, y_pred)
  r2 = r2_score(y_true, y_pred)
  d2 = d2_pinball_score(y_true, y_pred)
  mf_true = np.array([SNR_to_MF(x) for x in y_true])
  mf_pred = np.array([SNR_to_MF(x) for x in y_pred])
  under = np.sum(mf_true > mf_pred)
  over = np.sum(mf_true < mf_pred)
  correct = np.sum(mf_true == mf_pred)
  miss_rate = (over+under)/ (correct+over+under)

  results = {'Mean Squared Error':mse,
             'Mean Pinball Loss': pbl,
             'Pinball Loss At Alpha': pba,
             'Mean Absolute Error': mae,
             'R2 Square': r2,
             'D2 Pinball': d2,
             'Under Allocations': under,
             'Over Allocations': over,
             'Correct Allocations': correct,
             'Miss Rate': miss_rate}

  return results

In [ ]:
def eval_model_classifier(y_true1, y_pred1):
  y_true = np.array(y_true1, dtype=float)
  y_pred = np.array(y_pred1, dtype=float)
  accuracy = accuracy_score(y_true, y_pred)
  precision, recall, fscore, support = precision_recall_fscore_support(y_true, y_pred, zero_division = 0)
  under = np.sum(y_true > y_pred)
  over = np.sum(y_true < y_pred)
  correct = np.sum(y_true == y_pred)
  miss_rate = (over+under)/ (correct+over+under)
  results = {'Accuracy': accuracy,
              'Precision': precision.mean(),
              'Recall': recall.mean(),
              'FScore': fscore.mean(),
              'Under Allocations': under,
              'Over Allocations': over,
              'Correct Allocations': correct,
              'Miss Rate': miss_rate}

  return results

# Q025_pb0_e and Q075_pb1_g

In [ ]:
Q025_pb0_pred_e = Q025_pb0_e.predict(E_test)
Q075_pb1_pred_g = Q075_pb1_g.predict(G_test)

results_Q025_pb0_e = eval_model(e_test_snr, Q025_pb0_pred_e, 0)
results_Q075_pb1_g = eval_model(g_test_snr, Q075_pb1_pred_g, 1)

In [ ]:
print('Result of Q025_pb0_e:\n')
results_Q025_pb0_e

Result of Q025_pb0_e:



{'Mean Squared Error': 0.24993360437727133,
 'Mean Pinball Loss': 0.17480118722742713,
 'Pinball Loss At Alpha': 0.021648491426489676,
 'Mean Absolute Error': 0.34960237445485426,
 'R2 Square': 0.8911210625194014,
 'D2 Pinball': 0.714155702390653,
 'Under Allocations': 74,
 'Over Allocations': 2,
 'Correct Allocations': 291,
 'Miss Rate': 0.20708446866485014}

In [ ]:
print('Result of Q075_pb1_g:\n')
results_Q075_pb1_g

Result of Q075_pb1_g:



{'Mean Squared Error': 0.12946976716543293,
 'Mean Pinball Loss': 0.13750718624058614,
 'Pinball Loss At Alpha': 0.02543598355079497,
 'Mean Absolute Error': 0.2750143724811723,
 'R2 Square': 0.9238846435597083,
 'D2 Pinball': 0.7330344399269988,
 'Under Allocations': 2,
 'Over Allocations': 24,
 'Correct Allocations': 309,
 'Miss Rate': 0.07761194029850746}

# Q075_pb0_e and Q075_pb0_g

In [ ]:
Q075_pb0_pred_e = Q075_pb0_e.predict(E_test)
Q075_pb0_pred_g = Q075_pb0_g.predict(G_test)

results_Q075_pb0_e = eval_model(e_test_snr, Q075_pb0_pred_e, 0)
results_Q075_pb0_g = eval_model(g_test_snr, Q075_pb0_pred_g, 0)

In [ ]:
print('Result of Q075_pb0_e:\n')
results_Q075_pb0_e

Result of Q075_pb0_e:



{'Mean Squared Error': 0.06946272834958118,
 'Mean Pinball Loss': 0.10316860650364901,
 'Pinball Loss At Alpha': 0.12649601146041375,
 'Mean Absolute Error': 0.20633721300729801,
 'R2 Square': 0.9697398512054843,
 'D2 Pinball': 0.8312931489246572,
 'Under Allocations': 10,
 'Over Allocations': 18,
 'Correct Allocations': 339,
 'Miss Rate': 0.07629427792915532}

In [ ]:
print('Result of Q075_pb0_g:\n')
results_Q075_pb0_g

Result of Q075_pb0_g:



{'Mean Squared Error': 0.05470633453396327,
 'Mean Pinball Loss': 0.09004044686938381,
 'Pinball Loss At Alpha': 0.13947111076487323,
 'Mean Absolute Error': 0.18008089373876762,
 'R2 Square': 0.9678381119873815,
 'D2 Pinball': 0.8251895120182928,
 'Under Allocations': 4,
 'Over Allocations': 18,
 'Correct Allocations': 313,
 'Miss Rate': 0.06567164179104477}

# This is the testing for the Optimized Classifier

In [ ]:
clf_opt_e = load(join(model_path, 'opt150_classifier_e.joblib'))
clf_opt_g = load(join(model_path, 'opt150_classifier_g.joblib'))

g_test_mf = [str(SNR_to_MF(x)) for x in g_test_snr]
e_test_mf = [str(SNR_to_MF(x)) for x in e_test_snr]

In [ ]:
# Predict TEST for the chosen best models
clf_opt_pred_e = clf_opt_e.predict(E_test)
clf_opt_pred_g = clf_opt_g.predict(G_test)

results_clf_opt_e = eval_model_classifier(e_test_mf, clf_opt_pred_e)
results_clf_opt_g = eval_model_classifier(g_test_mf, clf_opt_pred_g)

In [ ]:
print('Result of clf_opt_e:\n')
results_clf_opt_e

Result of clf_opt_e:



{'Accuracy': 0.9373297002724795,
 'Precision': 0.9362929550715177,
 'Recall': 0.9321697465984564,
 'FScore': 0.9341666540772557,
 'Under Allocations': 11,
 'Over Allocations': 12,
 'Correct Allocations': 344,
 'Miss Rate': 0.06267029972752043}

In [ ]:
print('Result of clf_opt_g:\n')
results_clf_opt_g

Result of clf_opt_g:



{'Accuracy': 0.9373134328358209,
 'Precision': 0.9393318443756279,
 'Recall': 0.9357522089969909,
 'FScore': 0.9373714931524262,
 'Under Allocations': 8,
 'Over Allocations': 13,
 'Correct Allocations': 314,
 'Miss Rate': 0.0626865671641791}

# Saving the Results

In [ ]:
metrics_etest = [
    {'Model': 'Q025_pb0_e', **results_Q025_pb0_e},
    {'Model': 'Q075_pb0_e', **results_Q075_pb0_e},
]

all_metrics_etest = pd.DataFrame(metrics_etest)

metric_path = PATH + '/metrics/'

all_metrics_etest.to_csv(metric_path + 'all_metrics_etest.csv', index=False)

In [ ]:
metrics_gtest = [
    {'Model': 'Q075_pb1_g', **results_Q075_pb1_g},
    {'Model': 'Q075_pb0_g', **results_Q075_pb0_g},
]

all_metrics_gtest = pd.DataFrame(metrics_gtest)
all_metrics_gtest.to_csv(metric_path + 'all_metrics_gtest.csv', index=False)

In [ ]:
metrics_clftest = [
    {'Model': 'clf_opt_e', **results_clf_opt_e},
    {'Model': 'clf_opt_g', **results_clf_opt_g},
]

all_metrics_clftest = pd.DataFrame(metrics_clftest)

metric_path = PATH + '/metrics/'

all_metrics_clftest.to_csv(metric_path + 'all_metrics_clftest.csv', index=False)